In [ ]:
import csv
import monpa
from monpa import utils
from collections import Counter

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [ ]:
tf_counter = Counter() #統計 tf 用
df_counter = Counter() #統計 df 用

result = list() #儲存處理過的結果
f = open('./limit_down.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
i = 0
for row in rows: #對內容逐行列舉，row[0]是title, row[1]是content
    i = i+1 

    if(i > 1): #略過第一行:欄位名稱
        query = row[0]+' '+row[1]
        df_tmp = Counter() #暫存本篇 df 用
        str='' #暫存本篇斷詞後的內容用
        sentence_list = utils.short_sentence(query) #斷句
        for item in sentence_list:
            result_cut = monpa.cut(item) #斷詞
            for term in result_cut:
                term = term.strip() #去除前後多餘空白
                if(len(term) > 1): #若詞長>1
                    str=str+' '+term
                    tf_counter[term] += 1 #tf加1
                    if(df_tmp[term] == 0): #若本篇之前不曾出現
                        df_tmp[term]=1 #df標為1
    #     print(i, str)
        result.append(str)
        df_counter += df_tmp # 累加多篇df

f.close() #關閉檔案
#print(tf_counter) #這個字 在所有文章 出現幾次
#print(df_counter) #這個字 在多少篇文章有出現

/Users/peifish/opt/anaconda3/lib/python3.7/site-packages/monpa/crf_layer.py:374: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/TensorCompare.cpp:333.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


In [ ]:
total = 0
for word in df_counter:
    total += 1
print("dictionary size: ",total, "words")

dictionary size:  6165 words


In [ ]:
for term in df_counter.most_common(100): #印出df前100名
    print("[",term[0],term[1],"]")

[ 跌停 123 ]
[ 股價 89 ]
[ 台北 70 ]
[ 市場 70 ]
[ 記者 67 ]
[ 今天 57 ]
[ 報導 55 ]
[ 持續 49 ]
[ 時報 47 ]
[ 影響 47 ]
[ 表示 46 ]
[ 重挫 45 ]
[ 公司 41 ]
[ 下跌 39 ]
[ 2021年 39 ]
[ 跌幅 38 ]
[ 開盤 38 ]
[ 指出 38 ]
[ 目前 37 ]
[ 法人 36 ]
[ 預期 35 ]
[ 獲利 34 ]
[ 表現 34 ]
[ 賣壓 32 ]
[ 出現 32 ]
[ 指數 32 ]
[ 美國 32 ]
[ 營收 31 ]
[ 今年 31 ]
[ 台股 30 ]
[ 價格 30 ]
[ 中央社 29 ]
[ 超過 29 ]
[ 早盤 29 ]
[ 連續 28 ]
[ 交易 28 ]
[ 客戶 28 ]
[ 10 28 ]
[ 中國 28 ]
[ 財報 27 ]
[ 不過 27 ]
[ 由於 27 ]
[ 需求 27 ]
[ 以及 27 ]
[ 消息 26 ]
[ 族群 26 ]
[ KY 26 ]
[ 今日 25 ]
[ BR 25 ]
[ 來到 25 ]
[ 相關 25 ]
[ 上漲 25 ]
[ 主要 25 ]
[ 其中 25 ]
[ 財訊 25 ]
[ 受到 24 ]
[ 全球 24 ]
[ 公告 24 ]
[ 新台幣 24 ]
[ 投資人 24 ]
[ 成長 24 ]
[ 衝擊 24 ]
[ 大跌 24 ]
[ 以來 23 ]
[ 新聞 23 ]
[ 顯示 23 ]
[ 來源 23 ]
[ 去年 22 ]
[ 公布 22 ]
[ 外資 22 ]
[ 截至 22 ]
[ 資金 22 ]
[ 新高 22 ]
[ 終場 22 ]
[ 美元 22 ]
[ 增加 22 ]
[ 大幅 22 ]
[ 人民幣 22 ]
[ 高達 22 ]
[ 產品 22 ]
[ 未來 22 ]
[ 長榮 21 ]
[ 加上 21 ]
[ 營運 21 ]
[ 陽明 20 ]
[ 航運 20 ]
[ 三雄 20 ]
[ 漲幅 20 ]
[ 跌停板 20 ]
[ 類股 20 ]
[ 20 20 ]
[ 導致 20 ]
[ 走低 20 ]
[ 疫情 20 ]
[ 短期 20 ]
[ 投資 20 ]
[ 隨著 20 ]
[ 因此 20 ]
[ -KY 20 ]


In [ ]:
#檢驗是否全是中文字符
def is_all_chinese(strs):
    for _char in strs:
        if not '\u4e00' <= _char <= '\u9fa5':
            return False
    return True

In [ ]:
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords

In [ ]:
def hasNumber(stringVal):
    return any(elem.isdigit() for elem in stringVal)

In [ ]:
data = df_counter.items()
sorted_data = sorted(data, key=lambda tup: tup[1], reverse=True)

stopwords = stopwordslist('./stopwords.txt')

rank = []
in_stops = []
num_stops = []
for tup in sorted_data:
    if tup[0] in stopwords:
        in_stops.append(tup[0])
    elif hasNumber(tup[0]):
        num_stops.append(tup[0])
    else:
        rank.append(tup)
    if len(rank) == 100:
        break

print("Stopwords:", in_stops) 
print("number words:", num_stops) 
for i in range(100):
    print(i+1, rank[i])

Stopwords: ['不過', '由於', '以及', '其中', '截至', '加上', '隨著', '因此', '因為', '另外', '開始', '可能', '雖然', '其他']
number words: ['2021年', '10', '20', '6月']
1 ('跌停', 123)
2 ('股價', 89)
3 ('台北', 70)
4 ('市場', 70)
5 ('記者', 67)
6 ('今天', 57)
7 ('報導', 55)
8 ('持續', 49)
9 ('時報', 47)
10 ('影響', 47)
11 ('表示', 46)
12 ('重挫', 45)
13 ('公司', 41)
14 ('下跌', 39)
15 ('跌幅', 38)
16 ('開盤', 38)
17 ('指出', 38)
18 ('目前', 37)
19 ('法人', 36)
20 ('預期', 35)
21 ('獲利', 34)
22 ('表現', 34)
23 ('賣壓', 32)
24 ('出現', 32)
25 ('指數', 32)
26 ('美國', 32)
27 ('營收', 31)
28 ('今年', 31)
29 ('台股', 30)
30 ('價格', 30)
31 ('中央社', 29)
32 ('超過', 29)
33 ('早盤', 29)
34 ('連續', 28)
35 ('交易', 28)
36 ('客戶', 28)
37 ('中國', 28)
38 ('財報', 27)
39 ('需求', 27)
40 ('消息', 26)
41 ('族群', 26)
42 ('KY', 26)
43 ('今日', 25)
44 ('BR', 25)
45 ('來到', 25)
46 ('相關', 25)
47 ('上漲', 25)
48 ('主要', 25)
49 ('財訊', 25)
50 ('受到', 24)
51 ('全球', 24)
52 ('公告', 24)
53 ('新台幣', 24)
54 ('投資人', 24)
55 ('成長', 24)
56 ('衝擊', 24)
57 ('大跌', 24)
58 ('以來', 23)
59 ('新聞', 23)
60 ('顯示', 23)
61 ('來源', 23)
62 ('去年', 22)

In [ ]:
f = open('./limit_down_term.csv', 'w', newline='', encoding='utf-8') #以寫入模式開啟另一個檔
writer = csv.writer(f) #準備寫入
for term in tf_counter.most_common(100): #印出tf前100名
    writer.writerow(term) #寫入一行
f.close()